In [1]:
import osmnx as ox
from osmnx import projection
import pandana
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
target = [
    'rogers park', 'west ridge',
    'uptown', 'lincoln square',
    'north center', 'lake view', 'lincoln park',
    'near north side', 'edison park',
    'norwood park', 'jefferson park',
    'forest glen', 'north park', 'albany park',
    'portage park', 'irving park', 'dunning',
    'montclare', 'belmont cragin', 'hermosa',
    'avondale', 'logan square', 'humboldt park',
    'west town', 'austin', 'west garfield park',
    'east garfield park', 'near west side', 
    'north lawndale', 'south lawndale', 'lower west side', 'loop',
    'near south side', 'armour square', 'douglas',
    'oakland', 'fuller park', 'grand boulevard',
    'kenwood', 'washington park', 'hyde park',
    'woodlawn', 'south shore', 'chatham',
    'avalon park', 'south chicago', 'burnside',
    'calumet heights', 'roseland', 'pullman',
    'south deering', 'east side', 'west pullman',
    'riverdale', 'hegewisch', 'garfield ridge',
    'archer heights', 'brighton park', 'mckinley park',
    'bridgeport', 'new city', 'west elsdon',
    'gage park', 'clearing', 'west lawn',
    'chicago lawn', 'west englewood', 'englewood',
    'greater grand crossing', 'ashburn',
    'auburn gresham', 'beverly', 'washington heights',
    'mount greenwood', 'morgan park', 'ohare', 'edgewater'
]

target_upper = []
for i, neighborhood in enumerate(target): target_upper.append(neighborhood.upper())

In [3]:
tags = {
    'amenity' : [
        'cafe',
        'bar',
        'restaurant',
        'pub',
        'library',
        'school',
        'university',
        'atm',
        'cinema',
        'nightclub',
        'theatre',
        'police',
        'bench',
        'water_point'
    ],
    'shop' : [
        'bakery',
        'convenience',
        'supermarket',
        'mall',
        'shoes',
        'clothes',
        'department_store',
        'beverages',
        'ice_cream',
        'supermarket',
        'florist',
        'books',
        'anime',
        'laundry',
        'bicycle'
    ],
    'leisure' : [
        'fitness_centre',
        'amusement_arcade',
        'bowling_alley',
        'dog park',
        'fishing',
        'sports_hall',
        'fitness_station',
        'garden',
        'golf_course',
        'park'
    ]
}

In [4]:
# max finder for each col
from collections import deque
def findMax(x):
    queue = []
    max = 0
    for i, num in enumerate(x):
        if num > max:
            max = num
    return max

def findMin(x):
    queue = []
    min = 0
    for i, num in enumerate(x):
        if num > min:
            max = min
    return min

In [5]:
def inverse_distance_score(distances, max_possible_score=10):
    inverse_distances = 1 / distances
    score = inverse_distances.sum() / max_possible_score
    normalized_score = score / inverse_distances.size
    return normalized_score if normalized_score <= 1 else 1

In [6]:
hashmap = {
    'Community Name' : target_upper,
    'Walk_Score' : []
}

## Places of Interest (POI) Proximinity ##
The code below will perform a task where we will be calculating each neighborhood's arbitrary node's relative proximity to a place of interest. With the OSMnx defined tag above, we have set up the POIs which we have defined as being the metric/criteria of a city's walkability. For example, we will be calculating the walk-step based upon the node with other road networks. 

OSMnx documentation: https://osmnx.readthedocs.io/en/stable/

<style>
.text_cell_render {
font-family: Times New Roman, serif;
}
</style>

In [18]:
cityname = 'warsaw, poland'
crs = 3035
graph = ox.graph_from_place(cityname, network_type="walk")

# Project graph
graph = ox.projection.project_graph(graph, to_crs=crs)

In [8]:
for city in target_upper:
    if not city in ["BURNSIDE", "RIVERDALE", "MCKINLEY PARK"]:
        cityname = f"{city}, CHICAGO, ILLINOIS"
        crs = 3035
        graph = ox.graph_from_place(cityname, network_type='walk')
        graph = ox.projection.project_graph(graph, to_crs=crs)
        poi = ox.geometries.geometries_from_place(cityname, tags=tags)
        # project the place of interest
        poi = poi.to_crs(epsg=crs)

        max_time = 20
        walk_speed = 4.2
        for u, v, data in graph.edges(data=True):
            data['speed_kph'] = walk_speed
        graph = ox.add_edge_travel_times(graph)

    
        nodes = ox.graph_to_gdfs(graph, edges=False)[['x', 'y']]
        edges = ox.graph_to_gdfs(graph, nodes=False).reset_index()[['u', 'v', 'travel_time']]

        # Construct the pandana network model
        network = pandana.Network(
            node_x=nodes['x'],
            node_y=nodes['y'], 
            edge_from=edges['u'],
            edge_to=edges['v'],
            edge_weights=edges[['travel_time']]
        )
        centroids = poi.centroid
        """ 
        Calculate and specifiy a max travel distance
        for further implications onto a df we create
        """
        max_distance = max_time * 60 #mins
        network.set_pois(
            category='pois',
            maxdist = max_distance,
            maxitems = 10,
            x_col = centroids.x,
            y_col = centroids.y
        )

        distances = network.nearest_pois(
            distance=max_distance,
            category = 'pois',
            num_pois = 10
        )

        distances.astype(int)
        df = distances
        #(df.iloc[:, 1:] - df.iloc[:, 1:].min()) / (df.iloc[:, 1:].max() - df.iloc[:, 1:].min())
        X = (((df.sum() / df.count()).sum())/10)
        walk_score = X
        hashmap['Walk_Score'].append(walk_score)
    else:
        walk_score = 0
        hashmap['Walk_Score'].append(walk_score)
    

/var/folders/g4/ykwfgyl55t7dfzv_zc3dcshh0000gn/T/ipykernel_1472/1666489440.py:7: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release.
  poi = ox.geometries.geometries_from_place(cityname, tags=tags)


Generating contraction hierarchies with 8 threads.
Setting CH node vector of size 2263
Setting CH edge vector of size 7068
Range graph removed 7138 edges of 14136
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


/var/folders/g4/ykwfgyl55t7dfzv_zc3dcshh0000gn/T/ipykernel_1472/1666489440.py:7: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release.
  poi = ox.geometries.geometries_from_place(cityname, tags=tags)


Generating contraction hierarchies with 8 threads.
Setting CH node vector of size 3090
Setting CH edge vector of size 9804
Range graph removed 9946 edges of 19608
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
 100% 

/var/folders/g4/ykwfgyl55t7dfzv_zc3dcshh0000gn/T/ipykernel_1472/1666489440.py:7: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release.
  poi = ox.geometries.geometries_from_place(cityname, tags=tags)


...

Generating contraction hierarchies with 8 threads.
Setting CH node vector of size 2859
Setting CH edge vector of size 9426
Range graph removed 9534 edges of 18852
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


/var/folders/g4/ykwfgyl55t7dfzv_zc3dcshh0000gn/T/ipykernel_1472/1666489440.py:7: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release.
  poi = ox.geometries.geometries_from_place(cityname, tags=tags)


Generating contraction hierarchies with 8 threads.
Setting CH node vector of size 1628
Setting CH edge vector of size 4132
Range graph removed 4400 edges of 8264
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%
 100% 

/var/folders/g4/ykwfgyl55t7dfzv_zc3dcshh0000gn/T/ipykernel_1472/1666489440.py:7: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release.
  poi = ox.geometries.geometries_from_place(cityname, tags=tags)


Generating contraction hierarchies with 8 threads.
Setting CH node vector of size 2593
Setting CH edge vector of size 8686
Range graph removed 8792 edges of 17372
. 10% . 20% . 30% . 40% . 50% . 60% . 70% . 80% . 90% . 100%


In [13]:
import pandas as pd
len(hashmap['Community Name'])
len(hashmap["Walk_Score"][1:])
hashmap['Walk_Score'] = hashmap['Walk_Score'][1:]

In [14]:
df_poi = pd.DataFrame(hashmap)

In [23]:
# TARGET VALUES: BURNSIDE", "RIVERDALE", "MCKINLEY PARK", "OHARE"
mod = ['BURNSIDE', 'RIVERDALE', 'MCKINLEY PARK']
for i, val in enumerate(df_poi['Walk_Score']):
    if val in mod or val == 0.000000: 
        df_poi['Walk_Score'][i] = (df_poi['Walk_Score'].mean())
    if val == 'OHARE':
        df_poi['Walk_Score'][i] = 780.992313

/var/folders/g4/ykwfgyl55t7dfzv_zc3dcshh0000gn/T/ipykernel_1472/3677981211.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_poi['Walk_Score'][i] = (df_poi['Walk_Score'].mean())
/var/folders/g4/ykwfgyl55t7dfzv_zc3dcshh0000gn/T/ipykernel

In [32]:
def min_max_scale(series):
    # Calculate the minimum and maximum of the series
    min_val = series.min()
    max_val = series.max()
    scaled_series = (series - min_val) / (max_val - min_val)
    return scaled_series

df_poi['scaled_walkability'] = min_max_scale(df_poi['Walk_Score'])

In [41]:
df_poi.isnull().sum()
df_poi.to_csv('poi_data_processed.csv')
print(f'Saved df_poi to CSV format')

Saved df_poi to CSV format
